<a href="https://colab.research.google.com/github/warn4n/dl2025/blob/main/Notebooks/Chap13/13_4_Graph_Attention_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 13.4: Graph attention networks**

This notebook builds a graph attention mechanism from scratch, as discussed in section 13.8.6 of the book and illustrated in figure 13.12c

Work through the cells below, running each cell in turn. In various places you will see the words "TODO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.



In [1]:
import numpy as np
import matplotlib.pyplot as plt

The self-attention mechanism maps $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ and returns $N$ outputs $\mathbf{x}'_{n}\in \mathbb{R}^{D}$.  



In [2]:
# Set seed so we get the same random numbers
np.random.seed(1)
# Number of nodes in the graph
N = 8
# Number of dimensions of each input
D = 4

# Define a graph
A = np.array([[0,1,0,1,0,0,0,0],
              [1,0,1,1,1,0,0,0],
              [0,1,0,0,1,0,0,0],
              [1,1,0,0,1,0,0,0],
              [0,1,1,1,0,1,0,1],
              [0,0,0,0,1,0,1,1],
              [0,0,0,0,0,1,0,0],
              [0,0,0,0,1,1,0,0]]);
print(A)

# Let's also define some random data
X = np.random.normal(size=(D,N))

[[0 1 0 1 0 0 0 0]
 [1 0 1 1 1 0 0 0]
 [0 1 0 0 1 0 0 0]
 [1 1 0 0 1 0 0 0]
 [0 1 1 1 0 1 0 1]
 [0 0 0 0 1 0 1 1]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 1 1 0 0]]


We'll also need the weights and biases for the keys, queries, and values (equations 12.2 and 12.4)

In [3]:
# Choose random values for the parameters
omega = np.random.normal(size=(D,D))
beta = np.random.normal(size=(D,1))
phi = np.random.normal(size=(2*D,1))

We'll need a softmax operation that operates on the columns of the matrix and a ReLU function as well

In [4]:
# Define softmax operation that works independently on each column
def softmax_cols(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in) ;
  # Sum over columns
  denom = np.sum(exp_values, axis = 0);
  # Replicate denominator to N rows
  denom = np.matmul(np.ones((data_in.shape[0],1)), denom[np.newaxis,:])
  # Compute softmax
  softmax = exp_values / denom
  # return the answer
  return softmax


# Define the Rectified Linear Unit (ReLU) function
def ReLU(preactivation):
  activation = preactivation.clip(0.0)
  return activation


In [6]:
(np.concat([X[:,0], X[:,1]]) @ phi)

array([-3.00415955])

In [17]:
 # Now let's compute self attention in matrix form
def graph_attention(X,omega, beta, phi, A):

  # TODO -- Write this function (see figure 13.12c)
  # 1. Compute X_prime
  # 2. Compute S
  # 3. To apply the mask, set S to a very large negative number (e.g. -1e20) everywhere where A+I is zero
  # 4. Run the softmax function to compute the attention values
  # 5. Postmultiply X' by the attention values
  # 6. Apply the ReLU function
  # Replace this line:
  output = np.ones_like(X) ;
  X_prime = np.matmul(omega, X) + beta
  S = np.zeros((N,N))
  for m in range(N):
    for n in range(N):
      S[m,n] = (np.concat([X_prime[:,m], X_prime[:,n]]) @ phi)[0]
      if A[m,n] < 1 and m != n:
        S[m,n] = -1e20
  S = softmax_cols(S)
  #S = ReLU(S)


  output = np.matmul( X_prime, S)
  output = ReLU(output)

  return output;

In [16]:
# Test out the graph attention mechanism
np.set_printoptions(precision=3)
output = graph_attention(X, omega, beta, phi, A);
print("Correct answer is:")
print("[[0.    0.028 0.37  0.    0.97  0.    0.    0.698]")
print(" [0.    0.    0.    0.    1.184 0.    2.654 0.  ]")
print(" [1.13  0.564 0.    1.298 0.268 0.    0.    0.779]")
print(" [0.825 0.    0.    1.175 0.    0.    0.    0.  ]]]")


print("Your answer is:")
print(output)

[[0.986 0.848 0.    0.916 0.    0.    0.    0.   ]
 [0.011 0.009 0.063 0.01  0.056 0.    0.    0.   ]
 [0.    0.074 0.496 0.    0.443 0.    0.    0.   ]
 [0.003 0.002 0.    0.003 0.015 0.    0.    0.   ]
 [0.    0.066 0.441 0.071 0.394 0.059 0.    0.811]
 [0.    0.    0.    0.    0.034 0.005 0.005 0.069]
 [0.    0.    0.    0.    0.    0.927 0.995 0.   ]
 [0.    0.    0.    0.    0.058 0.009 0.    0.119]]
[[-1.161  1.349 -0.779  2.219  0.54   1.251 -2.634  1.241]
 [-0.741 -0.722  1.529  1.606 -2.451  5.392 -0.083  1.75 ]
 [ 1.359  0.138 -1.136  0.315  0.789  1.125 -2.885  0.376]
 [ 1.386 -1.828 -1.56  -0.949 -0.876  1.108 -2.266 -0.428]]
Correct answer is:
[[0.    0.028 0.37  0.    0.97  0.    0.    0.698]
 [0.    0.    0.    0.    1.184 0.    2.654 0.  ]
 [1.13  0.564 0.    1.298 0.268 0.    0.    0.779]
 [0.825 0.    0.    1.175 0.    0.    0.    0.  ]]]
Your answer is:
[[0.    0.    0.    0.    0.09  0.    0.    0.673]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [1.343 1.122

TODO -- Try to construct a dot-product self-attention mechanism as in practical 12.1 that respects the geometry of the graph and has zero attention between non-neighboring nodes by combining figures 13.12a and 13.12b.
